# Finger DetectionProject

## Finger Detection and Counting

## Imports

In [27]:
import cv2
import numpy as np
from sklearn.metrics import pairwise

### Global Variables

We will use these as we go along.

In [30]:
# This background will be a global variable that we update through a few functions
background = None

# Start with a halfway point between 0 and 1 of accumulated weight
accumulated_weight = 0.2

# Manually set up our ROI for grabbing the hand.
# Feel free to change these. I just chose the top right corner for filming.
roi_top = 20
roi_bottom = 300
roi_left = 300
roi_right = 600


## Finding Average Background Value

This function updates the background model by blending the current frame into the existing background using a weighted average.

The function calculates the weighted sum of the input image src and the accumulator dst so that dst becomes a running average of a frame sequence:

In [33]:
def calc_accum_avg(frame, accumulated_weight):
    '''
    Given a frame and a previous accumulated weight, computed the weighted average of the image passed in.
    '''

    # Grab the background
    global background
    
    # For first time, create the background from a copy of the frame.
    if background is None:
        background = frame.copy().astype('float')
        return None
        
    # compute weighted average, accumulate it and update the background
    cv2.accumulateWeighted(frame, background, accumulated_weight)


## Segment the Hand Region in Frame

This function detects the hand by comparing the current frame against the background.

Compute the absolute difference between the background and the current frame.
Apply a binary threshold to highlight the hand.
Extract contours from the thresholded image and return the largest contour, which corresponds to the hand.



In [36]:
def segment(frame, threshold_min=50):

    # Calculates the Absolute Difference between the background and the passed-in frame
    diff = cv2.absdiff(background.astype('uint8'), frame)

    # Apply a threshold to the image so we can grab the foreground
    ret, thresholded = cv2.threshold(diff, threshold_min, 255, cv2.THRESH_BINARY)

    # Grab the external contours from the image
    # Updated for newer OpenCV versions: only two return values
    contours, hierarchy = cv2.findContours(
        thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # If the length of contours list is 0, then we didn't grab any contours!
    if len(contours) == 0:
        return None
    else:
        # The largest external contour should be the hand
        hand_segment = max(contours, key=cv2.contourArea)

        # Return both the hand segment and the thresholded hand image
        return (thresholded, hand_segment)


## Counting Fingers with a Convex Hull

This function estimates the number of fingers based on the hand's contour.

Compute the convex hull of the hand segment.
Identify the extreme points of the convex hull to determine the hand's center and radius.
Create a circular Region of Interest (ROI) centered around the hand.
Count contours within the circular ROI, excluding potential noise (e.g., wrist).


Example of ConvexHulls:

<img src="hand_convex.png">

In [39]:
def count_fingers(thresholded, hand_segment):
    conv_hull = cv2.convexHull(hand_segment)

    # Extreme points
    top    = tuple(conv_hull[conv_hull[:, :, 1].argmin()][0])
    bottom = tuple(conv_hull[conv_hull[:, :, 1].argmax()][0])
    left   = tuple(conv_hull[conv_hull[:, :, 0].argmin()][0])
    right  = tuple(conv_hull[conv_hull[:, :, 0].argmax()][0])

    # Center of the hand
    cX = (left[0] + right[0]) // 2
    cY = (top[1] + bottom[1]) // 2

    # Euclidean distances from the center to the extreme points
    points = np.array([left, right, top, bottom])
    distance = pairwise.euclidean_distances([(cX, cY)], Y=points)[0]
    max_distance = distance.max()

    radius = int(0.8 * max_distance)
    circumference = 2 * np.pi * radius

    # Create circular ROI
    circular_roi = np.zeros(thresholded.shape[:2], dtype='uint8')

    # Draw the circular ROI
    cv2.circle(circular_roi, (cX, cY), radius, 255, 10)

    # Using bitwise AND to get the cutout of the hand
    circular_roi = cv2.bitwise_and(thresholded, thresholded, mask=circular_roi)

    # Find contours in circular ROI
    contours, hierarchy = cv2.findContours(
        circular_roi.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    count = 0
    for cnt in contours:
        (x, y, w, h) = cv2.boundingRect(cnt)

        # Check if the contour is not the wrist
        if (cY + (cY * 0.25)) > (y + h):
            # Limit the points
            if (circumference * 0.25) > cnt.shape[0]:
                count += 1

    return count


## Run Program
Convert the ROI to grayscale and blur it.
Update the background model if it's the first 60 frames.
After 60 frames, perform hand segmentation and count fingers.
Display the results on the frame and show additional debugging information (e.g., background model and difference image).


In [42]:
# Start capturing video from webcam
cam = cv2.VideoCapture(0)
num_frames = 0

while True:
    ret, frame = cam.read()
    if not ret:
        break

    frame_copy = frame.copy()

    # Flip the frame to avoid mirror image effect
    frame = cv2.flip(frame, 1)
    frame_copy = cv2.flip(frame_copy, 1)

    # Extract ROI
    roi = frame[roi_top:roi_bottom, roi_left:roi_right]

    # Convert ROI to grayscale and blur it
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 0)

    if num_frames < 60:
        calc_accum_avg(gray, accumulated_weight)
        if num_frames <= 59:
            cv2.putText(frame_copy, 'WAIT. GETTING BACKGROUND', (200, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
            cv2.imshow('Finger Count', frame_copy)
    else:
        hand = segment(gray)

        if hand is not None:
            thresholded, hand_segment = hand

            # Draw contours around hand segment
            cv2.drawContours(frame_copy, [hand_segment + (roi_left, roi_top)],
                             -1, (255, 0, 0), 5)

            fingers = count_fingers(thresholded, hand_segment)

            # Display finger count
            cv2.putText(frame_copy, str(fingers), (70, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            # Display thresholded image
            cv2.imshow('Thresholded', thresholded)

        else:
            cv2.putText(frame_copy, 'No hand detected', (200, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    # Draw ROI rectangle on the frame
    cv2.rectangle(frame_copy, (roi_left, roi_top),
                  (roi_right, roi_bottom), (0, 255, 0), 2)

    num_frames += 1

    # Display frames
    cv2.imshow('Finger Count', frame_copy)

    # Display background model and difference image for debugging
    if background is not None:
        cv2.imshow('Background Model', background.astype('uint8'))

    if num_frames >= 60:
        diff = cv2.absdiff(background.astype('uint8'), gray)
        cv2.imshow('Difference Image', diff)

    # Exit on pressing 'Esc'
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break

cam.release()
cv2.destroyAllWindows()
